In [129]:
# %load helper_functions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)

import seaborn as sns
import nltk
nltk.download('words')
pd.set_option('max_colwidth', 40)
import os
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
sws = set(stopwords.words('english'))
punctuation = set(string.punctuation)

def user_event_count():
    df['user_event_count'] = df.groupby('user_created').count().max(axis=1)
    return df


def previous_payouts(df):
    new_df=pd.DataFrame(df.previous_payouts[0])
    for i in range(1,df.previous_payouts.shape[0]):
        df_element=pd.DataFrame(df.previous_payouts[i])
        new_df=pd.concat([new_df, df_element])
    return new_df

def user_df():
    unique_cols = []
    for col in columns:
        if unique_col_indicator(df,col):
            unique_cols.append(col)
    user_unique_events = df.groupby('user_created').count().max(axis=1)
    user_unique_events = pd.DataFrame(user_unique_events,columns=['user_event_count'])
    return df[unique_cols].merge(user_unique_events,how='inner',on='user_created')

def unique_col_indicator(df,col):
    try:
        col_count = df.groupby(['user_created',col])['channels'].count().shape[0]
        user_count = df.groupby(['user_created'])['channels'].count().shape[0]
        if col_count == user_count:
            return True
        else:

            return False
    except:
        pass

    
#check
def user_unique_events_col():
    user_unique_events = df.groupby('user_created').count().max(axis=1)
    user_unique_events =  pd.DataFrame(user_unique_events,columns=['user_event_count'])
    return df.merge(user_unique_events,how='inner',on='user_created')

#check
def create_target():
    df['fraud'] = df['acct_type'].map({'fraudster_event': 1,
                                   'premium': 0,
                                   'spammer_warn': 0,
                                   'fraudster': 1,
                                   'spammer_limited': 0,
                                   'spammer_noinvite': 0,
                                   'locked': 1,
                                   'tos_lock': 0,
                                   'tos_warn': 0,
                                   'fraudster_att': 1,
                                   'spammer_web': 0,
                                   'spammer': 0})
    return df

from datetime import datetime

#check
def convert_date():
    df["approx_payout_date"] = pd.to_datetime(df["approx_payout_date"], unit = 's')
    df["event_created"] = pd.to_datetime(df["event_created"], unit = 's')
    df["event_end"] = pd.to_datetime(df["event_end"], unit = 's')
    df["event_start"] = pd.to_datetime(df["event_start"], unit = 's')
    df["event_published"] = pd.to_datetime(df["event_published"], unit = 's')
    return df

def low_cor_cols(r_score):
    df[num_cols()].corr()[['fraud']].values
    corr_df = df[num_cols()].corr()[['fraud']].sort_values('fraud')
    low_corr_mask = corr_df.sort_values('fraud').abs().lt(r_score).values.reshape(1,-1)[0]
    return corr_df[low_corr_mask]

def get_corrs():
    corr_df = df[num_cols()].corr()[['fraud']].sort_values('fraud')
    return corr_df

def num_cols():
    return df.describe().columns

def drop_cols():
    df1 = df.drop(low_cor_cols(.09).index.tolist(),axis=1)
    df2 = df1.drop(['acct_type','sale_duration','fraud','event_start',
                    'event_published','event_end','venue_state','ticket_types'],axis=1)
    
    return df2

#check
def description_cols():
    import clean_desc
    df['description'] = df['description'].apply(strip_tags)
    clean_desc(df)
    return df

#check
def ticket_types(df):
    new_df=pd.DataFrame(df.ticket_types[0])
    result_df=new_df[['quantity_sold', 'quantity_total', 'event_id']].groupby('event_id').sum()
    result_df=result_df.join(new_df[['availability', 'cost', 'event_id']].groupby('event_id').mean())

    for i in range(1,df.shape[0]):
        try:
            df_element=pd.DataFrame(df.ticket_types[i])
            grp_ele_df=df_element[['quantity_sold', 'quantity_total', 'event_id']].groupby('event_id').sum()
            grp_ele_df=grp_ele_df.join(df_element[['availability', 'cost', 'event_id']].groupby('event_id').mean())
            result_df=pd.concat([result_df, grp_ele_df])

        except:
            continue
    result_df.reset_index(inplace=True)
    result_df.columns=['object_id', 'quantity_sold', 'quantity_total', 'availability', 'cost']
    final_df=df.set_index('object_id').join(result_df.set_index('object_id'))
    final_df.reset_index(inplace=True)
    final_df[['quantity_sold', 'quantity_total','availability', 'cost']]=final_df[['quantity_sold', 'quantity_total','availability', 'cost']].fillna(0)
    return final_df

def timedelta(field):
    return field.days

def event_times():
    df['publish_time_to_start'] =  df['event_published'] - df['event_created'] 
    df['publish_time_to_start'] = df['publish_time_to_start'].apply(timedelta)
    return df

def has_venue_data():
    df['has_state'] = (df['venue_state'].fillna('None').replace('','None') == 'None').astype(int)
    df['has_country'] = (df['venue_country'].fillna('None').replace('','None') == 'None').astype(int)
    df['has_address'] = (df['venue_address'].fillna('None').replace('','None') == 'None').astype(int)
    df['has_venue_name'] = (df['venue_name'].fillna('None').replace('','None') == 'None').astype(int)
    return df

[nltk_data] Downloading package words to /home/ubuntu/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)

import seaborn as sns
import nltk
nltk.download('words')
pd.set_option('max_colwidth', 40)
import os
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
sws = set(stopwords.words('english'))
punctuation = set(string.punctuation)
from  helper_functions import user_event_count,\
                                ticket_types,\
                                previous_payouts,\
                                user_df,\
                                unique_col_indicator,\
                                create_target,\
                                convert_date,\
                                low_cor_cols,\
                                drop_cols,\
                                description_cols,\
                                ticket_types


[nltk_data] Downloading package words to /home/ubuntu/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /home/ubuntu/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [29]:
df = pd.read_json('data/data.json').drop('description',axis=1)
df = ticket_types(df)

In [35]:
df = convert_date()

In [31]:
df = create_target()

In [32]:
df = user_unique_events_col()

In [89]:
df = event_times()

In [94]:
num_cols()

Index(['object_id', 'body_length', 'channels', 'delivery_method',
       'fb_published', 'gts', 'has_analytics', 'has_header', 'has_logo',
       'name_length', 'num_order', 'num_payouts', 'org_facebook',
       'org_twitter', 'sale_duration', 'sale_duration2', 'show_map',
       'user_age', 'user_created', 'user_type', 'venue_latitude',
       'venue_longitude', 'quantity_sold', 'quantity_total', 'availability',
       'cost', 'fraud', 'user_event_count', 'event_duration',
       'publish_time_to_start'],
      dtype='object')

In [120]:
drop_cols()

,approx_payout_date,body_length,channels,country,currency,delivery_method,email_domain,event_created,fb_published,has_logo,listed,name,name_length,org_desc,org_facebook,org_name,org_twitter,payee_name,payout_type,previous_payouts,sale_duration2,ticket_types,user_age,user_created,user_type,venue_address,venue_country,venue_name,venue_state,availability,event_duration
0,2010-02-13 12:00:00,3852,5,US,USD,0.0,gmail.com,2010-01-06 01:01:46,0,0,y,"99 HOUR ""NO SLEEP"" SUPER BOWL CELEBR...",60,,0.0,Party Starz Ent & Diverse Int'l Group,0.0,,,[],33,"[{'event_id': 527017, 'cost': 25.0, ...",36,1259613950,1,717 Washington Avenue,US,INK Nightclub - South Beach,FL,1.0,33
1,2012-01-19 14:00:00,3561,8,US,USD,0.0,gmail.com,2011-12-28 19:21:18,0,1,y,THE KING OF R & B MEETS THE KING OF ...,90,,0.0,,0.0,,ACH,"[{'name': 'PARTY STARZ ENT INC', 'cr...",16,"[{'event_id': 2687179, 'cost': 50.0,...",758,1259613950,1,17800 NE 5TH AVE,US,,,1.0,16
2,2011-10-13 12:00:00,533,8,US,USD,0.0,gmail.com,2011-10-07 04:59:28,0,1,y,R.KELLY LIVE UPCLOSE AND PERSONAL @ ...,88,"<p><strong></strong><span class=""tex...",17.0,PARTY STARZ & BUGATTE WORLD ENT,14.0,,,[],1,"[{'event_id': 2296426, 'cost': 0.0, ...",675,1259613950,1,1045 5th street,US,CLUB PLAY,FL,1.0,1
3,2012-06-21 11:00:00,630,8,US,USD,0.0,gmail.com,2012-05-07 20:56:17,0,1,y,"2NIGHT! 2NIGHT! ""X-RATED"" WEDNESDAY'...",67,,0.0,"PARTY STARZ ENT INC,",0.0,,,[],9,"[{'event_id': 3500119, 'cost': 10.0,...",889,1259613950,1,2700 NW 167 ST,US,PALM GARDENS,FL,1.0,39
4,2012-06-02 11:00:00,990,5,US,USD,0.0,gmail.com,2012-05-01 22:02:25,0,1,y,MR.WOBBLE BABY!*ALL-WHITE* GROWN & S...,89,,0.0,"THE ALL NEW ""PALM GARDENS""",0.0,,ACH,"[{'name': 'PARTY STARZ ENT INC', 'cr...",26,"[{'event_id': 3464783, 'cost': 10.0,...",883,1259613950,1,2700 NW 167 ST,US,PALM GARDENS,FL,1.0,26
5,2011-12-18 12:00:00,1478,8,US,USD,0.0,gmail.com,2011-12-03 03:28:41,0,1,y,CHRISETTE MICHELLE CELEBRITY BIRTHDA...,59,"<p><strong></strong><span class=""tex...",17.0,PARTY STARZ & BUGATTE WORLD ENT,14.0,,,[],10,"[{'event_id': 2589732, 'cost': 20.0,...",732,1259613950,1,8358 PINES BLVD,US,CAFE IGUANAS,FL,1.0,10
6,2012-05-06 06:30:00,8836,6,US,USD,0.0,gmail.com,2012-04-27 19:21:37,0,1,y,2NITE! 2NITE! HEAT VS KNICKS PLAYOFF...,188,"<table style=""width: 421px; height: ...",0.0,PALM GARDENS,0.0,,,[],3,"[{'event_id': 3441863, 'cost': 20.0,...",879,1259613950,1,2700 NW 167 ST,,,,1.0,3
7,2012-08-10 13:00:00,649,8,US,USD,0.0,gmail.com,2012-07-19 05:13:38,0,1,y,Rick! Ross! Album Release! Party & C...,171,"<p><span style=""font-size: medium;"">...",0.0,NELSON INC.....,0.0,,,[],17,"[{'event_id': 3939892, 'cost': 0.0, ...",961,1259613950,1,17800 NE 5TH,,KING OF DIAMONDS,,1.0,17
8,2012-01-06 12:00:00,593,8,US,USD,0.0,gmail.com,2011-12-20 17:54:31,0,1,y,ATTENTION! ATTENTION! NEW LOCATION F...,165,"<p><strong></strong><span class=""tex...",17.0,PARTY STARZ & BUGATTE WORLD ENT,14.0,,,[],11,"[{'event_id': 2629715, 'cost': 25.0,...",750,1259613950,1,12389 PEMBROKE ROAD,US,GRAND CAFE BAR & GRILL,FL,1.0,11
9,2010-02-13 18:00:00,45882,8,US,USD,0.0,gmail.com,2009-11-30 20:45:54,1,1,y,"99 HOUR ""NO SLEEP"" SUPER BOWL CELEBR...",72,,0.0,Party Starz Ent & Diverse Int'l Group,0.0,,,[],66,"[{'event_id': 501276, 'cost': 250.0,...",0,1259613950,1,6701 COLLINS AVE,US,DEAUVILLE BEACH RESORT,,1.0,69


In [128]:
def has_venue_data():
    df['has_state'] = (df['venue_state'].fillna('None').replace('','None') == 'None').astype(int)
    df['has_country'] = (df['venue_country'].fillna('None').replace('','None') == 'None').astype(int)
    df['has_address'] = (df['venue_address'].fillna('None').replace('','None') == 'None').astype(int)
    df['has_venue_name'] = (df['venue_name'].fillna('None').replace('','None') == 'None').astype(int)
    return df

0        0
1        1
2        0
3        0
4        0
5        0
6        1
7        1
8        0
9        1
10       0
11       0
12       1
13       0
14       1
15       0
16       0
17       0
18       0
19       1
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
        ..
14307    1
14308    0
14309    0
14310    0
14311    1
14312    0
14313    0
14314    0
14315    0
14316    0
14317    1
14318    1
14319    1
14320    0
14321    0
14322    0
14323    0
14324    0
14325    0
14326    0
14327    0
14328    0
14329    0
14330    0
14331    0
14332    0
14333    0
14334    0
14335    0
14336    0
Name: venue_state, Length: 14337, dtype: int64